# Level 1: Year/Month Heatmap

In [29]:
import pandas as pd
import plotly.graph_objects as go

def load_data(csv_path):
    """Load CSV data into a DataFrame."""
    return pd.read_csv(csv_path)

def prepare_data(df):
    """Convert date column to datetime and extract year and month."""
    df["date"] = pd.to_datetime(df["date"])
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    return df

def aggregate_monthly(df):
    """Group data by year and month to compute monthly max and min temperatures,
    along with the corresponding dates."""
    grouped = df.groupby(["year", "month"])

    max_data = grouped.apply(lambda g: pd.Series({
        "max_temp": g["max_temperature"].max(),
        "max_date": g.loc[g["max_temperature"].idxmax(), "date"].strftime("%Y-%m-%d")
    })).reset_index()

    min_data = grouped.apply(lambda g: pd.Series({
        "min_temp": g["min_temperature"].min(),
        "min_date": g.loc[g["min_temperature"].idxmin(), "date"].strftime("%Y-%m-%d")
    })).reset_index()

    return max_data, min_data

def pivot_data(max_data, min_data):
    """Pivot the aggregated data to form matrices with months as rows and years as columns."""
    max_temp_matrix = max_data.pivot(index="month", columns="year", values="max_temp").sort_index()
    max_date_matrix = max_data.pivot(index="month", columns="year", values="max_date").sort_index()
    min_temp_matrix = min_data.pivot(index="month", columns="year", values="min_temp").sort_index()
    min_date_matrix = min_data.pivot(index="month", columns="year", values="min_date").sort_index()
    return max_temp_matrix, max_date_matrix, min_temp_matrix, min_date_matrix

def create_hover_text(max_temp_matrix, max_date_matrix, min_temp_matrix, min_date_matrix):
    """Generate hover text for both maximum and minimum temperature matrices."""
    hover_text_max = []
    hover_text_min = []

    for month in max_temp_matrix.index:
        row_hover_max = []
        row_hover_min = []
        for year in max_temp_matrix.columns:
            temp_max = max_temp_matrix.loc[month, year]
            date_max = max_date_matrix.loc[month, year]
            hover_max = f"Month: {month}<br>Year: {year}<br>Date: {date_max}<br>Max Temp: {temp_max}"
            row_hover_max.append(hover_max)

            temp_min = min_temp_matrix.loc[month, year]
            date_min = min_date_matrix.loc[month, year]
            hover_min = f"Month: {month}<br>Year: {year}<br>Date: {date_min}<br>Min Temp: {temp_min}"
            row_hover_min.append(hover_min)
        hover_text_max.append(row_hover_max)
        hover_text_min.append(row_hover_min)

    return hover_text_max, hover_text_min

def create_heatmap_trace(matrix, hover_text):
    """Create a Plotly heatmap trace from the provided matrix and hover text."""
    trace = go.Heatmap(
        x=matrix.columns.astype(str),  # x-axis: Years
        y=matrix.index.astype(str),      # y-axis: Months
        z=matrix.values,
        text=hover_text,
        hoverinfo="text",
        colorscale="OrRd",
        colorbar=dict(title="Temperature"),
        xgap=2,  # gap between columns
        ygap=2   # gap between rows
    )
    return trace

def build_figure(trace_max, trace_min):
    """Build the final figure with toggle buttons to switch between max and min views."""
    fig = go.Figure(data=[trace_max, trace_min])

    # Initially, show only the maximum temperature trace.
    fig.data[0].visible = True
    fig.data[1].visible = False

    # Add toggle buttons to switch views.
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="right",
                buttons=[
                    dict(
                        label="Max Temperature",
                        method="update",
                        args=[{"visible": [True, False]},
                              {"title": "Monthly Maximum Temperature"}]
                    ),
                    dict(
                        label="Min Temperature",
                        method="update",
                        args=[{"visible": [False, True]},
                              {"title": "Monthly Minimum Temperature"}]
                    )
                ],
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.0,
                xanchor="left",
                y=1.15,
                yanchor="top"
            )
        ],
        title="Monthly Maximum Temperature of Hong Kong",
        xaxis_title="Year",
        yaxis_title="Month"
    )

    return fig

csv_path = "temperature_daily.csv"

# Load and prepare the data.
df = load_data(csv_path)
df = prepare_data(df)

# Aggregate data monthly.
max_data, min_data = aggregate_monthly(df)

# Pivot data to create matrices.
max_temp_matrix, max_date_matrix, min_temp_matrix, min_date_matrix = pivot_data(max_data, min_data)

# Create hover text.
hover_text_max, hover_text_min = create_hover_text(max_temp_matrix, max_date_matrix,
                                                       min_temp_matrix, min_date_matrix)

# Create heatmap traces.
trace_max = create_heatmap_trace(max_temp_matrix, hover_text_max)
trace_min = create_heatmap_trace(min_temp_matrix, hover_text_min)

# Build and show the figure.
fig = build_figure(trace_max, trace_min)
fig.show()

<ipython-input-29-46c7111f9742>:20: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

<ipython-input-29-46c7111f9742>:25: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



# Level 2 Challenge: Improvement of the Year/Month Heatmap

In [27]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

def load_and_prepare_data(csv_path, start_year=2008, end_year=2017):
    df = pd.read_csv(csv_path)
    df["date"] = pd.to_datetime(df["date"])
    df = df[(df["date"].dt.year >= start_year) & (df["date"].dt.year <= end_year)]
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    return df

def aggregate_monthly(df):
    agg_max = df.groupby(["year", "month"])["max_temperature"].max().reset_index()
    agg_min = df.groupby(["year", "month"])["min_temperature"].min().reset_index()
    agg_max_dict = {(row["year"], row["month"]): row["max_temperature"] for idx, row in agg_max.iterrows()}
    agg_min_dict = {(row["year"], row["month"]): row["min_temperature"] for idx, row in agg_min.iterrows()}
    return agg_max, agg_min, agg_max_dict, agg_min_dict

def get_color_from_value(v, vmin, vmax, colorscale):
    norm = (v - vmin) / (vmax - vmin) if vmax > vmin else 0.5
    idx = int(round(norm * (len(colorscale)-1)))
    idx = max(0, min(idx, len(colorscale)-1))
    return colorscale[idx]

def create_background_shapes(fig, years, months, agg_dict, vmin, vmax, colorscale, opacity=0.7):
    shapes = []
    n_cols = len(years)
    for i, month in enumerate(months, start=1):
        for j, year in enumerate(years, start=1):
            subplot_index = (i - 1) * n_cols + j
            if subplot_index == 1:
                xaxis_id = "xaxis"
                yaxis_id = "yaxis"
            else:
                xaxis_id = f"xaxis{subplot_index}"
                yaxis_id = f"yaxis{subplot_index}"
            try:
                x_domain = fig.layout[xaxis_id].domain
                y_domain = fig.layout[yaxis_id].domain
            except Exception:
                continue
            agg_val = agg_dict.get((year, month), None)
            fillcolor = get_color_from_value(agg_val, vmin, vmax, colorscale) if agg_val is not None else "rgba(0,0,0,0)"
            shape = dict(
                type="rect",
                xref="paper",
                yref="paper",
                x0=x_domain[0],
                x1=x_domain[1],
                y0=y_domain[0],
                y1=y_domain[1],
                fillcolor=fillcolor,
                opacity=opacity,
                layer="below",
                line_width=0
            )
            shapes.append(shape)
    return shapes

def create_dummy_trace(vmin, vmax, title, colorscale, colorbar_x):
    dummy = go.Scatter(
        x=[0],
        y=[0],
        mode="markers",
        marker=dict(
            size=10,
            color=[(vmin + vmax) / 2],
            colorscale=colorscale,
            cmin=vmin,
            cmax=vmax,
            showscale=True,
            colorbar=dict(title=title, x=colorbar_x)
        ),
        showlegend=False,
        hoverinfo="none",
        opacity=0
    )
    return dummy

def build_figure(df, agg_max, agg_min, agg_max_dict, agg_min_dict):
    colorscale = px.colors.sequential.OrRd
    years = sorted(df["year"].unique())
    months = list(range(1, 13))
    n_cols = len(years)
    n_rows = len(months)

    fig = make_subplots(
        rows=n_rows, cols=n_cols,
        shared_xaxes=False, shared_yaxes=False,
        horizontal_spacing=0.02, vertical_spacing=0.02,
        start_cell="top-left",
        row_titles=[f"Month {m}" for m in months],
        column_titles=[str(year) for year in years]
    )

    # Plot mini line charts.
    for i, month in enumerate(months, start=1):
        for j, year in enumerate(years, start=1):
            df_cell = df[(df["year"] == year) & (df["month"] == month)]
            if df_cell.empty:
                continue
            days = df_cell["date"].dt.day
            fig.add_trace(
                go.Scatter(
                    x=days,
                    y=df_cell["max_temperature"],
                    mode="lines+markers",
                    line=dict(color="darkorange"),
                    name="Max Temperature" if (i == 1 and j == 1) else None,
                    showlegend=(i == 1 and j == 1),
                    hovertemplate="Day: %{x}<br>Max Temp: %{y}<extra></extra>"
                ),
                row=i, col=j
            )
            fig.add_trace(
                go.Scatter(
                    x=days,
                    y=df_cell["min_temperature"],
                    mode="lines+markers",
                    line=dict(color="red"),
                    name="Min Temperature" if (i == 1 and j == 1) else None,
                    showlegend=(i == 1 and j == 1),
                    hovertemplate="Day: %{x}<br>Min Temp: %{y}<extra></extra>"
                ),
                row=i, col=j
            )

    # Background shapes.
    vmin_max, vmax_max = agg_max["max_temperature"].min(), agg_max["max_temperature"].max()
    vmin_min, vmax_min = agg_min["min_temperature"].min(), agg_min["min_temperature"].max()

    shapes_max = create_background_shapes(fig, years, months, agg_max_dict, vmin_max, vmax_max, colorscale)
    shapes_min = create_background_shapes(fig, years, months, agg_min_dict, vmin_min, vmax_min, colorscale)

    fig.update_layout(shapes=shapes_max)

    # Dummy traces for color bars.
    dummy_max = create_dummy_trace(vmin_max, vmax_max, "Max Temp", colorscale, colorbar_x=1.15)
    dummy_min = create_dummy_trace(vmin_min, vmax_min, "Min Temp", colorscale, colorbar_x=1.15)

    N = len(fig.data)  # number of mini line chart traces
    fig.add_trace(dummy_max)
    fig.add_trace(dummy_min)

    # Toggle buttons.
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="right",
                buttons=[
                    dict(
                        label="Max Background",
                        method="update",
                        args=[
                            {"visible": [True] * N + [True, False]},
                            {"shapes": shapes_max, "title": "Daily Temperature with Max Background"}
                        ]
                    ),
                    dict(
                        label="Min Background",
                        method="update",
                        args=[
                            {"visible": [True] * N + [False, True]},
                            {"shapes": shapes_min, "title": "Daily Temperature with Min Background"}
                        ]
                    )
                ],
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.0,
                xanchor="left",
                y=1.02,
                yanchor="top"
            )
        ]
    )

    fig.update_layout(
        height=1200,
        width=1600,
        title="Daily Temperature Changes (Max & Min) with Background (2008–2017)",
        showlegend=True,
        margin=dict(r=150)
    )

    # Optionally, hide tick labels.
    for key in fig['layout']:
        if key.startswith("xaxis") or key.startswith("yaxis"):
            fig['layout'][key].update(showticklabels=False)

    return fig

# --- Main Execution ---
csv_path = "temperature_daily.csv"
df3 = load_and_prepare_data(csv_path)
agg_max, agg_min, agg_max_dict, agg_min_dict = aggregate_monthly(df3)
fig = build_figure(df3, agg_max, agg_min, agg_max_dict, agg_min_dict)
fig.show()